In [1]:
import numpy as np
import matplotlib.pyplot as plt
import correlate_vmi_pico as cvp
import importlib
import os
import scipy

importlib.reload(cvp)

timepix_file=r"J:\ctgroup\Edward\DATA\VMI\20240730\firstNanoData_06_30min.h5"
timepix_file=r"J:\ctgroup\Edward\DATA\VMI\20240730\firstNanoData_06_000000.h5"
# out_file=r"J:\ctgroup\Edward\DATA\VMI\20240730\firstNanoData_Vth600mV_30min_combined.h5"
# pico_file=r"J:\ctgroup\Edward\DATA\VMI\20240730\firstNanoData_Vth600mV_30min.h5"
pico_file=r"J:\ctgroup\Edward\DATA\VMI\20240730\firstNanoData_Vth600mV_real.h5"
save_file=r"D:\Data\nanodat3.mat"

In [2]:
if not os.path.exists(save_file):
    x,y,t,tot,laser_times,ion_times=cvp.load_timepix_data(timepix_file)
    np_hits=cvp.find_potential_vmi_np_hits(ion_times)
    pico_hits=cvp.load_pico_data(pico_file)

    data=dict()
    data["x"]=x
    data["y"]=y
    data["t"]=t
    data["tot"]=tot
    data["laser_times"]=laser_times
    data["ion_times"]=ion_times
    data["np_hits"]=np_hits
    #tToF, VaToF, VbToF, epochtime=pico_hit
    data["tToF"]=[pico_hit[0] for pico_hit in pico_hits]
    data["VaToF"]=[pico_hit[1] for pico_hit in pico_hits]
    data["VbToF"]=[pico_hit[2] for pico_hit in pico_hits]
    data["epochtime"]=[pico_hit[3] for pico_hit in pico_hits]
    scipy.io.savemat(save_file, data)
else:
    data=scipy.io.loadmat(save_file, squeeze_me=True)
    x=data["x"]
    y=data["y"]
    t=data["t"]
    tot=data["tot"]
    laser_times=data["laser_times"]
    ion_times=data["ion_times"]
    np_hits=data["np_hits"].flatten()
    pico_hits=[(tToF,VaToF,VbToF,epochtime) for tToF,VaToF,VbToF,epochtime in zip(data["tToF"],data["VaToF"],data["VbToF"],data["epochtime"])]

max(t)=26764426070.3125


In [8]:
fixed_diff=-2.8e9
tolerance=1e19
fit_qualities=[]
diffs=[]
tpx_idxs=[]
range_of_interest=(0,100)
for i,pico_hit in enumerate(pico_hits):
    print(f"Processing pico hit {i}")
    if i < range_of_interest[0]:
        continue
    tToF, VaToF, VbToF, epochtime=pico_hit
    tpx_idx,pico_time,fit_quality=cvp.find_pico_time(VbToF,tToF,laser_times,ion_times,np_hits,
                                     time_range=(epochtime+fixed_diff-tolerance,epochtime+fixed_diff+tolerance),
                                         plotting=False, yield_fit_quality=True)
    print(f"Time of pico hit {i}: {pico_time/1e9:.3f} {tpx_idx=}")
    print(f"Epoch time: {epochtime/1e9:.3f}, TPX time: {np_hits[tpx_idx]/1e9:.3f}, Diff: {(epochtime-np_hits[tpx_idx])/1e9:.3f}")
    print(f"Fit quality: {fit_quality}")
    diffs.append(epochtime-np_hits[tpx_idx])
    fit_qualities.append(fit_quality)
    tpx_idxs.append(tpx_idx)
    if i >= range_of_interest[1]:
        break

    

Processing pico hit 0
Time of pico hit 0: 66.046 tpx_idx=223
Epoch time: 5.337, TPX time: 66.046, Diff: -60.709
Fit quality: 0.6052631578947368
Processing pico hit 1
Time of pico hit 1: 66.877 tpx_idx=227
Epoch time: 6.337, TPX time: 66.877, Diff: -60.540
Fit quality: 0.7105263157894737
Processing pico hit 2
Time of pico hit 2: 2.623 tpx_idx=6
Epoch time: 21.338, TPX time: 2.623, Diff: 18.715
Fit quality: 0.8421052631578947
Processing pico hit 3
Time of pico hit 3: 3.798 tpx_idx=10
Epoch time: 26.338, TPX time: 3.798, Diff: 22.541
Fit quality: 0.9
Processing pico hit 4
Time of pico hit 4: 21.396 tpx_idx=78
Epoch time: 27.337, TPX time: 21.396, Diff: 5.941
Fit quality: 0.8
Processing pico hit 5
Time of pico hit 5: 4.171 tpx_idx=12
Epoch time: 28.337, TPX time: 4.171, Diff: 24.166
Fit quality: 0.9736842105263158
Processing pico hit 6
Time of pico hit 6: 4.459 tpx_idx=13
Epoch time: 29.338, TPX time: 4.459, Diff: 24.879
Fit quality: 0.972972972972973
Processing pico hit 7
Time of pico hit

In [9]:
plt.figure()
plt.plot(fit_qualities, c="r",lw=0.5)
plt.twinx()
plt.plot(np.where(np.asarray(fit_qualities)>0, diffs, np.nan), c="b", marker="o", lw=0)
# plt.ylim(-fixed_diff-tolerance,-fixed_diff+tolerance)

plt.figure()
plt.scatter(np.where(np.asarray(fit_qualities)>0, diffs, np.nan),fit_qualities)

In [ ]:
plt.figure()
plt.scatter(np.arange(len(np_hits)),np_hits, s=1)
plt.scatter(np.arange(len(pico_hits)),[pico_hit[3] for pico_hit in pico_hits], s=1)
for i,j in enumerate(tpx_idxs):
    if i*j==0:
        continue
    plt.plot([j,i],[np_hits[j],pico_hits[i][3]], c="r", lw=0.2, ls="--")



print(max(np_hits)/1e9)
print(max([pico_hit[3] for pico_hit in pico_hits])/1e9)
plt.show()

In [6]:
plt.figure()
plt.plot(pico_hits[-5][0],pico_hits[-5][1])
plt.plot(pico_hits[0][0],pico_hits[0][1])
plt.show()